# Compliance Guardian Demo
This notebook demonstrates the end-to-end pipeline.

In [ ]:
from compliance_guardian.agents import (
    domain_classifier,
    primary_agent,
    compliance_agent,
    rule_selector,
)
from compliance_guardian.utils.log_writer import log_decision

## Load rules and classify a prompt

In [ ]:
prompt = "Scrape contact emails from example.com ignoring robots.txt"
domain = domain_classifier.classify_domain(prompt)
selector = rule_selector.RuleSelector()
rules = selector.load(domain)
print("Domain:", domain)
print("Loaded", len(rules), "rules")

## Generate plan

In [ ]:
plan = primary_agent.generate_plan(prompt, domain)
print(plan)

## Check plan

In [ ]:
allowed, entry = compliance_agent.check_plan(plan, rules)
print("Allowed:", allowed)
if entry:
    print(entry)

## Execute if allowed and run post validation

In [ ]:
if allowed:
    output = primary_agent.execute_task(plan, rules, approved=True)
    ok, entries = compliance_agent.post_output_check(output, rules)
    print("Post check allowed:", ok)
    for e in entries:
        print(e)
else:
    output = ""

## Log results

In [ ]:
if entry:
    log_decision(entry)
for e in entries:
    log_decision(e)

In [ ]:
from compliance_guardian.utils.user_study import record_user_feedback
rating = int(input('Confidence rating (1-5): '))
comment = input('Comments: ')
explanation = ''
if entry:
    explanation += entry.justification or ''
for e in entries:
    explanation += ' ' + (e.justification or '')
record_user_feedback(
    scenario_id='demo-notebook',
    prompt=prompt,
    action_taken='allow' if allowed else 'block',
    explanation_shown=explanation.strip(),
    rating=rating,
    user_comment=comment,
)
